Импортируем библиотеки

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier, log_evaluation, early_stopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

Перейдем к построению моделей.

In [ ]:
df = pd.read_csv('/final_data.csv')

X = df.drop('category', axis=1)
y = df['category']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

cat_model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.03,
    depth=7,
    l2_leaf_reg=7,
    random_seed=42,
    early_stopping_rounds=50,
    random_strength=3,
    loss_function='MultiClass',
    eval_metric='TotalF1',
    devices='GPU',
    verbose=100,
    bootstrap_type='Bernoulli',
    subsample=0.6
)

cat_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100, plot=True)

cat_pred_train = cat_model.predict(X_train)
cat_pred_val = cat_model.predict(X_val)

cat_model_acc = accuracy_score(y_train, cat_pred_train)
cat_model_precision = precision_score(y_train, cat_pred_train, average='weighted')
cat_model_recall = recall_score(y_train, cat_pred_train, average='weighted')

cat_model_confusion = confusion_matrix(y_train, cat_pred_train)
class_name = ['DDoS', 'DoS', 'Benign', 'Spoofing']

fig, axes = plt.subplots(1, 3, figsize=(15,4))
disp = ConfusionMatrixDisplay(confusion_matrix=cat_model_confusion, display_labels=class_name)
disp.plot(ax=axes[0], cmap='Blues')
axes[0].set_title('Стандартная матрица ошибок')
disp.plot()
plt.show()

print(f'Accuracy: {cat_model_acc:.3f}')
print(f'Precision: {cat_model_precision:.3f}')
print(f'Recall: {cat_model_recall:.3f}')